In [1]:
VERSION = '99_v1_01_02'

In [2]:
model_list = [    
    {'path':'../../13_Baseline5/exp/result/13_v1_03/oof_df.csv', 'w':1}, #deberta-v3-base, seed100, split00_v2_07
    {'path':'../../13_Baseline5/exp/result/13_v1_04/oof_df.csv', 'w':1}, #deberta-v3-base, seed200, split00_v2_07
    {'path':'../../13_Baseline5/exp/result/13_v1_07/oof_df.csv', 'w':1}, #deberta-base, seed100, split00_v2_07
]

In [3]:
import pandas as pd

stack_df = None
for i,model in enumerate(model_list):
    pred_df = pd.read_csv(model['path'])
    pred_df = pred_df.rename(columns={
        'pred_ineffective':f'Ineffective_{i}', 
        'pred_adequate':f'Adequate_{i}', 
        'pred_effective':f'Effective_{i}'
    })
    if stack_df is None:
        stack_df = pred_df.copy()
    else:
        stack_df = stack_df.merge(pred_df[['discourse_id',f'Ineffective_{i}',f'Adequate_{i}',f'Effective_{i}']],
                                  on='discourse_id', how='left')

train_df = pd.read_csv('../../input/feedback-prize-effectiveness/train.csv')
oof_df = train_df[['discourse_id']].merge(stack_df, on='discourse_id', how='left')

In [4]:
for col_name in ['Ineffective','Adequate','Effective']:
    cols = [f'{col_name}_{i}' for i in range(len(model_list))]
    oof_df[col_name] = oof_df[cols].mean(axis=1)

In [5]:
from sklearn.metrics import log_loss

score = log_loss(oof_df['label'].values, oof_df[['Ineffective','Adequate','Effective']].values)
print('oof={:.4f}'.format(score))

oof=0.6042
